# Transfer Learning If You Have Small images, to change the Last layer of class and trained the Model with Own Custom Data

In [4]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [21]:
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224)) ## set image size
x = image.img_to_array(img)
print (x.shape)
x = np.expand_dims(x, axis=0)
print (x.shape)
x = preprocess_input(x)
print('Input image shape:', x.shape)

(224, 224, 3)
(1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)


In [29]:
# Loading the training data
PATH = os.getcwd()
# Define data path
data_path = PATH + '\\data'
data_dir_list = os.listdir(data_path)

In [30]:
data_dir_list

['cats',
 'data.zip',
 'dogs',
 'horses',
 'Humans',
 'imagenet_utils.py',
 'README.md',
 'resnet50.py',
 'transfer_learning_resnet50_custom_data.py',
 'transfer_learning_test.py',
 'transfer_learning_vgg16_custom_data.py',
 'vgg16.py']

In [31]:
img_data_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'\\'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '\\'+ dataset + '\\'+ img
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #x = x/255
        print('Input image shape:', x.shape)
        img_data_list.append(x)

Loaded the images of dataset-cats

Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)
Input image shape: (1, 224, 2

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\jitendra.dubey\\Others\\ML_Syllabus\\My Files\\data\\data.zip'

In [32]:
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(202, 1, 224, 224, 3)
(1, 202, 224, 224, 3)
(202, 224, 224, 3)


In [34]:
# Define the number of classes
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:202]=0
labels[202:404]=1
labels[404:606]=2
labels[606:]=3

names = ['cats','dogs','horses','humans']

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [35]:
labels[0:202]=0

# Only Class Name has changed from Layers

In [38]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

# downloading the models if not found
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet') ## include top =True means add the all conV feature 
# FC level only class of the FC has changed
model.summary()
last_layer = model.get_layer('fc2').output ### last layer remove and added our "classes"


553476096/553467096 [==============================] - 150s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                          

In [39]:
#x= Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)

In [40]:
custom_vgg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [41]:
## freeze other layers only one f2 layer is changed,this is called feature based extracter fine tuning
## Very Important Line, Freezing the layers [] 
for layer in custom_vgg_model.layers[:-1]:
	layer.trainable = False

custom_vgg_model.layers[3].trainable

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


t=time.time()
#	t = now()
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Epoch 1/12
6/6 [==============================] - 24s 4s/step - loss: 1.1084 - accuracy: 0.8012 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 2/12
6/6 [==============================] - 29s 5s/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/12
6/6 [==============================] - 31s 5s/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 3.6275e-04 - val_accuracy: 1.0000
Epoch 4/12
6/6 [==============================] - 35s 6s/step - loss: 7.1041e-04 - accuracy: 1.0000 - val_loss: 3.2103e-04 - val_accuracy: 1.0000
Epoch 5/12
6/6 [==============================] - 34s 6s/step - loss: 6.1712e-04 - accuracy: 1.0000 - val_loss: 2.7734e-04 - val_accuracy: 1.0000
Epoch 6/12
6/6 [==============================] - 34s 6s/step - loss: 5.2480e-04 - accuracy: 1.0000 - val_loss: 2.3373e-04 - val_accuracy: 1.0000
Epoch 7/12
6/6 [==============================] - 37s 6s/step - loss: 4.3544e-04 - accuracy: 1.0000 - val_loss: 1.6860e-04 - val_accuracy: 1.000

# FC layes with fc1 and fc2 also been Changed

CHanges layes list:

    1- block5_pool
    2- fc1
    3- fc2

In [42]:
#Training the feature extraction also

image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

model.summary()

last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)
custom_vgg_model2.summary()

# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
	layer.trainable = False

custom_vgg_model2.summary()

custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

t=time.time()
#	t = now()
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

#%%
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(12)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

6/6 [==============================] - 25s 4s/step - loss: 24.8233 - accuracy: 0.0000e+00 - val_loss: 23.9453 - val_accuracy: 0.0000e+00
Epoch 2/12
6/6 [==============================] - 32s 5s/step - loss: 23.2535 - accuracy: 0.0000e+00 - val_loss: 22.5657 - val_accuracy: 0.0000e+00
Epoch 3/12
6/6 [==============================] - 32s 5s/step - loss: 21.8127 - accuracy: 0.0000e+00 - val_loss: 21.2784 - val_accuracy: 0.0000e+00
Epoch 4/12
6/6 [==============================] - 32s 5s/step - loss: 20.4713 - accuracy: 0.0000e+00 - val_loss: 20.1351 - val_accuracy: 0.0000e+00
Epoch 5/12
6/6 [==============================] - 33s 5s/step - loss: 19.2827 - accuracy: 0.0000e+00 - val_loss: 19.0384 - val_accuracy: 0.0000e+00
Epoch 6/12
6/6 [==============================] - 34s 6s/step - loss: 18.1478 - accuracy: 0.0000e+00 - val_loss: 18.0036 - val_accuracy: 0.0000e+00
Epoch 7/12
6/6 [==============================] - 34s 6s/step - loss: 17.0650 - accuracy: 0.0062 - val_loss: 17.0216 - val_

KeyError: 'acc'